In [1]:
import zipfile
import scipy.io as sio
import io
from typing import Dict, Any, Optional

def load_mat_from_zip(zip_path: str) -> Optional[Dict[str, Any]]:
    with zipfile.ZipFile(zip_path, 'r') as zf:
        mat_file_name = next((name for name in zf.namelist() if name.endswith('.mat')), None)
        
        if not mat_file_name:
            print(f"在 {zip_path} 中未找到.mat文件。")
            return None
        
        mat_bytes = zf.read(mat_file_name)
        mat_stream = io.BytesIO(mat_bytes)
        mat_contents = sio.loadmat(mat_stream)
        return mat_contents


In [2]:
# --- 使用示例 ---
zip_file = 'test_data/Patient_1_preictal_segment_0002.mat.zip'
mat_data = load_mat_from_zip(zip_file)
var_name = list(mat_data.keys())[-1]
print(f"var_name: {var_name}")

print(f"data: {mat_data[var_name][0][0][0].shape}")
print(f"data_length_sec: {mat_data[var_name][0][0][1][0][0]}")
print(f"sampling_frequency: {mat_data[var_name][0][0][2][0][0]}")
print(f"channels: {mat_data[var_name][0][0][3][0]}")
print(f"sequence: {mat_data[var_name][0][0][4][0][0]}")

var_name: preictal_segment_2
data: (15, 3000000)
data_length_sec: 600
sampling_frequency: 5000
channels: [array(['LD_1'], dtype='<U4') array(['LD_3'], dtype='<U4')
 array(['LD_4'], dtype='<U4') array(['LD_5'], dtype='<U4')
 array(['LD_6'], dtype='<U4') array(['LD_7'], dtype='<U4')
 array(['LD_8'], dtype='<U4') array(['RD_1'], dtype='<U4')
 array(['RD_2'], dtype='<U4') array(['RD_3'], dtype='<U4')
 array(['RD_4'], dtype='<U4') array(['RD_5'], dtype='<U4')
 array(['RD_6'], dtype='<U4') array(['RD_7'], dtype='<U4')
 array(['RD_8'], dtype='<U4')]
sequence: 2


In [3]:
import mne

def load_mat_to_mne(file_path, ch_type, target_sfreq=None):
    data_bundle = load_mat_from_zip(file_path)
    var_name = list(data_bundle.keys())[-1]
    data = data_bundle[var_name][0][0][0] * 1e-6
    sf = data_bundle[var_name][0][0][2][0][0]
    channels = [arr[0] for arr in data_bundle[var_name][0][0][3][0]]
    num_channels = len(channels)

    info = mne.create_info(ch_names=channels, sfreq=sf, ch_types=[ch_type] * num_channels)
    raw = mne.io.RawArray(data, info, verbose=False)
    if target_sfreq is not None and target_sfreq != sf:
        raw.resample(target_sfreq, verbose=False)
    return raw

In [4]:
import numpy as np
from seizure_detector import HDCClassifier, PostProcessor

interictal_train_file = 'test_data/Patient_1_interictal_segment_0001.mat.zip'
interictal_train_raw = load_mat_to_mne(interictal_train_file, 'seeg', 500)
num_channels = len(interictal_train_raw.ch_names)

print(f"sf: {interictal_train_raw.info['sfreq']}")
print(f"num_channels: {num_channels}")
print(f"channels: {interictal_train_raw.ch_names}")
print(f"interictal_train_data: {interictal_train_raw.get_data().shape}")

# 我只找到了含有 preictal 标注的数据，假设其是 ictal
ictal_train_file = 'test_data/Patient_1_preictal_segment_0001.mat.zip'
ictal_train_raw = load_mat_to_mne(ictal_train_file, 'seeg', 500)

sf: 500.0
num_channels: 15
channels: ['LD_1', 'LD_3', 'LD_4', 'LD_5', 'LD_6', 'LD_7', 'LD_8', 'RD_1', 'RD_2', 'RD_3', 'RD_4', 'RD_5', 'RD_6', 'RD_7', 'RD_8']
interictal_train_data: (15, 300000)


In [5]:
ictal_train_data = ictal_train_raw.get_data()
interictal_train_data = interictal_train_raw.get_data()
sf = interictal_train_raw.info['sfreq']
num_channels = len(interictal_train_raw.ch_names)

classifier = HDCClassifier(num_channels=num_channels, tr_percentile=20)
post_processor = PostProcessor(tc_threshold=7)

classifier.train(ictal_train_data, 'ictal', sf)
classifier.train(interictal_train_data, 'interictal', sf)
classifier.calculate_tr_threshold([ictal_train_data], sf)
post_processor.tr_threshold = classifier.tr_threshold

Updating Ictal prototype...
Updating Interictal prototype...
Calculating confidence threshold (tr) using 20th percentile...
Confidence threshold (tr) automatically set to: 2763.00


In [61]:
# Ictal Test
for i in range(1, 6):
    ictal_test_file = f'test_data/Patient_1_preictal_segment_000{i}.mat.zip'
    ictal_test_raw = load_mat_to_mne(ictal_test_file, 'seeg', 500)
    ictal_test_data = ictal_test_raw.get_data()
    
    clip_results = classifier.classify_clip(ictal_test_data, sf)
    final_decision = post_processor.process_clip_results(clip_results)
    print(f"  - Ictal {i}: Final Decision -> {final_decision}")

  - Ictal 1: Final Decision -> SEIZURE
  - Ictal 2: Final Decision -> SEIZURE
  - Ictal 3: Final Decision -> SEIZURE
  - Ictal 4: Final Decision -> SEIZURE
  - Ictal 5: Final Decision -> SEIZURE


In [62]:
# Interictal Test
for i in range(1, 6):
    interictal_test_file = f'test_data/Patient_1_interictal_segment_000{i}.mat.zip'
    interictal_test_raw = load_mat_to_mne(interictal_test_file, 'seeg', 500)
    interictal_test_data = interictal_test_raw.get_data()
    
    clip_results = classifier.classify_clip(interictal_test_data, sf)
    final_decision = post_processor.process_clip_results(clip_results)
    print(f"  - Interictal {i}: Final Decision -> {final_decision}")

  - Interictal 1: Final Decision -> NON-SEIZURE
  - Interictal 2: Final Decision -> NON-SEIZURE
  - Interictal 3: Final Decision -> NON-SEIZURE
  - Interictal 4: Final Decision -> NON-SEIZURE
  - Interictal 5: Final Decision -> NON-SEIZURE
